# AI Capabilities Identification: From Intent to Implementation

This notebook demonstrates how to trace from **user intent** → **AI tasks** → **required capabilities** → **intrinsic implementations**.

**What you'll learn:**
- How to use LLMs to identify AI tasks from use case descriptions
- How to discover which capabilities are needed for those tasks using AtlasExplorer
- How to find which model intrinsics/adapters implement those capabilities
- Complete end-to-end traceability for AI system design

## 1. Setup and Imports

In [1]:
from ai_atlas_nexus import AIAtlasNexus
from ai_atlas_nexus.blocks.ai_atlas_explorer import AtlasExplorer
from ai_atlas_nexus.blocks.inference import OllamaInferenceEngine

/Users/elizabeth.daly/Documents/Dev/ai-atlas-nexus/src/ai_atlas_nexus/toolkit/job_utils.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## 2. Initialize AI Atlas Nexus and Explorer

Load the ontology with all capabilities, tasks, and intrinsics.

In [2]:
# Initialize AI Atlas Nexus
ran = AIAtlasNexus()

# Initialize AtlasExplorer for navigating the knowledge graph
explorer = AtlasExplorer(ran._ontology)

print("✓ AI Atlas Nexus initialized")
print(f"  Loaded {len(explorer.get_all_capabilities())} capabilities")
print(f"  Loaded {len(explorer.get_all_tasks())} AI tasks")
print(f"  Loaded {len(ran._ontology.llmintrinsics or [])} LLM intrinsics")
print(f"  Loaded {len(ran._ontology.adapters or [])} adapters")

[2025-11-28 17:05:13:195] - INFO - AIAtlasNexus - Created AIAtlasNexus instance. Base_dir: None


✓ AI Atlas Nexus initialized
  Loaded 11 capabilities
  Loaded 8 AI tasks
  Loaded 9 LLM intrinsics
  Loaded 19 adapters


## 3. Configure Inference Engine (Optional)

**This section is optional** - only needed for LLM-based task identification from use case descriptions.

**Options:**
- **Ollama** (local) - Free, requires Ollama running locally
- **WML** (IBM watsonx.ai) - Requires API credentials
- **vLLM** (self-hosted) - Requires vLLM server

**Don't have an inference engine?** Skip to Section 5 to work with predefined tasks.

In [3]:
# Configure Ollama inference engine
# Change model_name_or_path to a model you have installed (run: ollama list)

inference_engine = OllamaInferenceEngine(
    model_name_or_path="granite3.2:8b",  # Change to your preferred model
    credentials={"api_url": "http://localhost:11434"},
    parameters={"temperature": 0.1, "num_predict": 500}
)

print("✓ Inference engine configured")
print(f"  Model: granite3.2:8b")
print(f"  Endpoint: http://localhost:11434")

[2025-11-28 17:05:16:886] - INFO - AIAtlasNexus - OLLAMA inference engine will execute requests on the server at http://localhost:11434.
[2025-11-28 17:05:17:12] - INFO - AIAtlasNexus - Created OLLAMA inference engine.


✓ Inference engine configured
  Model: granite3.2:8b
  Endpoint: http://localhost:11434


## 4. Example 1: RAG Use Case - Customer Support

Let's analyze a real-world RAG use case and identify what AI tasks and capabilities are needed.

In [4]:
# Define the use case
usecase = """Generate personalized, relevant responses and recommendations for 
customer support agents. The system should summarize customer claims, answer 
questions about products, and provide contextual suggestions to help agents 
resolve issues faster."""

print("Use Case:")
print(usecase)
print("\n" + "="*80)

Use Case:
Generate personalized, relevant responses and recommendations for 
customer support agents. The system should summarize customer claims, answer 
questions about products, and provide contextual suggestions to help agents 
resolve issues faster.



### Step 1: Identify AI Tasks from Use Case Description

Use the LLM to automatically identify which AI tasks are needed.

In [5]:
# Use AI Atlas Nexus to identify tasks
task_predictions = ran.identify_ai_tasks_from_usecases(
    usecases=[usecase],
    inference_engine=inference_engine,
)

identified_tasks = task_predictions[0].prediction

print("\nIdentified AI Tasks:")
for i, task in enumerate(identified_tasks, 1):
    print(f"  {i}. {task}")
print("\n" + "="*80)

Inferring with OLLAMA:   0%|          | 0/1 [00:00<?, ?it/s]


Identified AI Tasks:
  1. Text-to-Image
  2. Text-to-Speech
  3. Text Generation
  4. Summarization
  5. Question Answering
  6. Text Classification



### Step 2: Explore Available Capabilities

Let's see what capabilities are defined in the IBM AI Capabilities Framework.

In [6]:
# Get all capabilities using AtlasExplorer
capabilities = explorer.get_all_capabilities()

print(f"\nIBM AI Capabilities Framework: {len(capabilities)} capabilities\n")

# Group by capability group
for cap in sorted(capabilities, key=lambda x: x.isPartOf):
    print(f"• {cap.name}")
    print(f"  ID: {cap.id}")
    print(f"  Group: {cap.isPartOf}")
    if hasattr(cap, 'description') and cap.description:
        # Truncate long descriptions
        desc = cap.description[:100] + "..." if len(cap.description) > 100 else cap.description
        print(f"  Description: {desc}")
    print()


IBM AI Capabilities Framework: 11 capabilities

• Code Generation
  ID: ibm-cap-code-generation
  Group: ibm-cap-group-code
  Description: The ability to write code in programming languages based on natural language descriptions, including...

• Code Understanding & Explanation
  ID: ibm-cap-code-understanding
  Group: ibm-cap-group-code
  Description: The ability to read, understand, and explain code, including summarizing code functionality, identif...

• Reading Comprehension
  ID: ibm-cap-reading-comprehension
  Group: ibm-cap-group-comprehension
  Description: The ability to understand written text, extract key information, and answer questions about the cont...

• Contextual Understanding
  ID: ibm-cap-contextual-understanding
  Group: ibm-cap-group-comprehension
  Description: The ability to understand context, resolve coreferences, and interpret implicit information in text....

• Domain Knowledge
  ID: ibm-cap-domain-knowledge
  Group: ibm-cap-group-domain-knowledge
  Descrip

### Step 3: Map Tasks to Capabilities

Now we can query which capabilities are required for each task using the AtlasExplorer API.

In [7]:
# Example: Get capabilities for Question Answering task
qa_capabilities = explorer.get_capabilities_for_task(task_id="question-answering")

print("Question Answering task requires these capabilities:\n")
for cap in qa_capabilities:
    print(f"  • {cap.name}")
    print(f"    ID: {cap.id}")
    if hasattr(cap, 'description') and cap.description:
        desc = cap.description[:120] + "..." if len(cap.description) > 120 else cap.description
        print(f"    Description: {desc}")
    print()

print(f"\n✓ Found {len(qa_capabilities)} capabilities required for Question Answering")

Question Answering task requires these capabilities:

  • Contextual Understanding
    ID: ibm-cap-contextual-understanding
    Description: The ability to understand context, resolve coreferences, and interpret implicit information in text. Essential for multi...

  • Domain Knowledge
    ID: ibm-cap-domain-knowledge
    Description: Possessing and applying specialized knowledge across technical, scientific, and professional domains including medicine,...

  • Factual Accuracy
    ID: ibm-cap-factual-accuracy
    Description: The ability to produce factually correct information and avoid hallucinations, confabulations, or false statements. Esse...

  • Logical Reasoning
    ID: ibm-cap-logical-reasoning
    Description: The ability to perform deductive, inductive, and formal logical inference. Includes syllogistic reasoning, propositional...

  • Reading Comprehension
    ID: ibm-cap-reading-comprehension
    Description: The ability to understand written text, extract key information

### Step 4: Find Intrinsics that Implement Capabilities

For each capability, we can find which intrinsics and adapters implement it.

In [8]:
# Example: Get intrinsics for Reading Comprehension capability
intrinsics = explorer.get_intrinsics_for_capability(
    capability_id="ibm-cap-reading-comprehension"
)

print("Reading Comprehension is implemented by:\n")
for intr in intrinsics:
    print(f"  • {intr.name}")
    print(f"    ID: {intr.id}")
    print()

print(f"\n✓ Found {len(intrinsics)} implementations for Reading Comprehension")

Reading Comprehension is implemented by:

  • Granite 3.3 8b Instruct - Answerability Determination
    ID: ibm-factuality-adapter-granite-3.3-8b-instruct-lora-answerability-determination

  • Granite 3.3 8b Instruct - Context Relevance
    ID: ibm-factuality-adapter-granite-3.3-8b-instruct-lora-context-relevance

  • Granite 3.3 8b Instruct - Query Rewrite
    ID: ibm-factuality-adapter-granite-3.3-8b-instruct-lora-query-rewrite

  • Granite 3.3 Instruct - Passage Reranking
    ID: ibm-factuality-adapter-granite-3.3-instruct-lora-passage-reranking


✓ Found 4 implementations for Reading Comprehension


### Step 5: End-to-End Tracing

We can trace from a task all the way to intrinsics in one call.

In [9]:
# Trace from task → capabilities → intrinsics
trace_result = explorer.trace_task_to_intrinsics(task_id="question-answering")

print("Complete Trace for Question Answering:\n")
print(f"Task: {trace_result['task'].name}")
print(f"\nRequired Capabilities ({len(trace_result['capabilities'])}):\n")

for cap in trace_result['capabilities']:
    print(f"  • {cap.name}")
    
    # Show intrinsics for this capability
    cap_intrinsics = trace_result['intrinsics_by_capability'].get(cap.id, [])
    if cap_intrinsics:
        print(f"    Implemented by {len(cap_intrinsics)} intrinsic(s):")
        for intr in cap_intrinsics[:2]:  # Show first 2
            print(f"      - {intr.name}")
        if len(cap_intrinsics) > 2:
            print(f"      ... and {len(cap_intrinsics) - 2} more")
    print()

print(f"\n✓ Total intrinsics across all capabilities: {len(trace_result['all_intrinsics'])}")

Complete Trace for Question Answering:

Task: Question Answering

Required Capabilities (5):

  • Contextual Understanding
    Implemented by 5 intrinsic(s):
      - Granite 3.3 8b Instruct - Answerability Determination
      - Granite 3.3 8b Instruct - Context Relevance
      ... and 3 more

  • Domain Knowledge

  • Factual Accuracy

  • Logical Reasoning

  • Reading Comprehension


✓ Total intrinsics across all capabilities: 5


## 5. Example 2: Direct Task Analysis (No LLM Required)

You can also work directly with known tasks without using LLM inference.

In [10]:
# Get all available tasks using AtlasExplorer
tasks = explorer.get_all_tasks()

print(f"Available AI Tasks: {len(tasks)}\n")

for task in tasks:
    print(f"• {task.name}")
    print(f"  ID: {task.id}")
    if hasattr(task, 'description') and task.description:
        desc = task.description[:150] + "..." if len(task.description) > 150 else task.description
        print(f"  Description: {desc}")
    print()

Available AI Tasks: 8

• Question Answering
  ID: question-answering
  Description: Question Answering models can retrieve the answer to a question from a given text, which is useful for searching for an answer in a document. Some que...

• Summarization
  ID: summarization
  Description: Summarization is the task of producing a shorter version of a document while preserving its important information. Some models can extract text from t...

• Text Classification
  ID: text-classification
  Description: Text Classification is the task of assigning a label or class to a given text. Some use cases are sentiment analysis, natural language inference, and ...

• Text Generation
  ID: text-generation
  Description: Generating text is the task of generating new text given another text. These models can, for example, fill in incomplete text or paraphrase.

• Translation
  ID: translation
  Description: Translation converts a sequence of text from one language to another. It is one of several ta

## 6. Coverage Analysis

Analyze which capabilities have task and intrinsic mappings.

In [11]:
# Analyze coverage
all_capabilities = explorer.get_all_capabilities()
all_tasks = explorer.get_all_tasks()

# Count capabilities that have task mappings
caps_with_tasks = 0
caps_with_intrinsics = 0

for cap in all_capabilities:
    # Check if capability has task mappings
    tasks_for_cap = explorer.get_tasks_for_capability(capability_id=cap.id)
    if tasks_for_cap:
        caps_with_tasks += 1
    
    # Check if capability has intrinsic implementations
    intrinsics_for_cap = explorer.get_intrinsics_for_capability(capability_id=cap.id)
    if intrinsics_for_cap:
        caps_with_intrinsics += 1

print("="*80)
print("IBM AI CAPABILITIES FRAMEWORK - COVERAGE ANALYSIS")
print("="*80)

print(f"\nTotal Capabilities: {len(all_capabilities)}")
print(f"Total Tasks: {len(all_tasks)}")
print(f"\nCapabilities with task mappings: {caps_with_tasks}/{len(all_capabilities)}")
print(f"Capabilities with intrinsic implementations: {caps_with_intrinsics}/{len(all_capabilities)}")

print("\n" + "="*80)

IBM AI CAPABILITIES FRAMEWORK - COVERAGE ANALYSIS

Total Capabilities: 11
Total Tasks: 8

Capabilities with task mappings: 9/11
Capabilities with intrinsic implementations: 6/11



## 7. Summary

### What This Notebook Demonstrated:

✅ **Use Case → AI Tasks**: Using LLM inference to identify relevant AI tasks from natural language descriptions

✅ **Capability Taxonomy**: Exploring the IBM AI Capabilities Framework (11 capabilities across 5 domains)

✅ **Task→Capability Mappings**: Using AtlasExplorer to query which capabilities are required for tasks

✅ **Capability→Intrinsic Mappings**: Finding which models/adapters implement capabilities

✅ **End-to-End Tracing**: Complete traceability from task to implementation

### Key AtlasExplorer Methods:

- `explorer.get_all_capabilities()` - Get all capability definitions
- `explorer.get_all_tasks()` - Get all AI task definitions
- `explorer.get_capabilities_for_task(task_id)` - Find required capabilities for a task
- `explorer.get_intrinsics_for_capability(capability_id)` - Find implementations for a capability
- `explorer.trace_task_to_intrinsics(task_id)` - Complete end-to-end trace

### Key Benefits:

- **Complete Traceability**: From business use case → technical implementation
- **Standardized Vocabulary**: Consistent capability definitions across the organization
- **Gap Analysis**: Identify which capabilities lack implementations
- **Model Selection**: Choose models based on required capabilities

---

**For more information:**
- AtlasExplorer: See `atlas_explorer_usage.ipynb`
- AI Atlas Nexus: https://github.com/IBM/ai-atlas-nexus